## Счёты

--------------------------------------------------------------------------------
Данная тетрадь является попыткой смоделировать вычисление корня
*Вавилонским методом* на счётах применяя операции *НОД*, *НОК* и *факторизацию*
исключительно счётным целочисленным методом.

In [1]:
import numpy as np
import pandas as pd

In [2]:
class Row():
    def __init__(self, period, name='a'):
        # Размер порядка системы счисления
        # Для десятичной системы нужно передать число 10
        # При этом максимальное количество элементов будет 9
        self.period = period
        self.data = []
        self.stack = []
        self.carry = False
        self.name = name

    def preserve(self):
        self.stack.append(self.data.copy())

    def restore(self):
        assert(len(self.stack) > 0)
        self.data = self.stack.pop()
    
    def hasItems(self):
        assert(len(self.data) < self.period)
        # Длина по идее не может быть меньше нуля
        # Стоит проверять только в случае, если длина
        # ожидается более нуля
        # assert(len(self.data) >= 0)        
        return len(self.data) > 0

    def hasSpace(self):
        assert(len(self.data) < self.period)
        # assert(len(self.data) >= 0)        
        return len(self.data) < self.period - 1
    
    def isEmpty(self):
        assert(len(self.data) < self.period)
        return len(self.data) == 0

    def isFull(self):
        assert(len(self.data) < self.period)
        return len(self.data) == self.period - 1

    def isCarry(self):
        return self.carry
    
    # Больше чем переданная строка
    def isBigger(self, row):
        assert(type(row) == Row)
        assert(self is not row)
        assert(self.period == row.period)

        row.clearCarry()
        row.preserve()
        row.sub(self)
        row.restore()

        return row.isCarry()

    def isEquals(self, row):
        assert(type(row) == Row)
        assert(self is not row)
        assert(self.period == row.period)

        row.clearCarry()
        row.preserve()
        row.sub(self)
        row.restore()

        if (row.isCarry()):
            return False

        self.clearCarry()
        self.preserve()
        self.sub(row)
        self.restore()

        return not self.isCarry()
    
    def clear(self):
        self.data.clear()
        self.carry = False

    def clearCarry(self):
        self.carry = False
    
    def fill(self):
        assert(len(self.data) < self.period)
        assert(not self.carry)
        self.carry = True
        while(not self.isFull()):
            self.data.append(1)

    def take(self):
        assert(len(self.data) < self.period)
        assert(not self.carry)
        self.carry = True
        self.data.clear()
        

    def push(self):
        assert(len(self.data) < self.period - 1)
        self.data.append(1)
    
    def pop(self):
        assert(len(self.data) < self.period)
        assert(len(self.data) > 0)
        # На счётах могут быть только камушки
        assert(self.data.pop() == 1)

    def pushOrCarry(self):
        assert(len(self.data) < self.period)
        assert(not self.carry)

        if (self.hasSpace()):
            self.push()
        else:
            self.take()
    
    def popOrCarry(self):
        assert(len(self.data) < self.period)
        assert(not self.carry)

        if (self.hasItems()):
            self.pop()
        else:
            self.fill()

    # Эта операция может произойти в состоянии заёма
    # При повторном заёме произойдёт ошибка
    def add(self, row):
        assert(type(row) == Row)
        assert(self is not row)
        assert(self.period == row.period)
        assert(len(self.data) < self.period)

        if (row.isEmpty()):
            return
        
        row.preserve()
        while(row.hasItems()):
            row.pop()
            if (self.isFull()):
                self.take()
            else:
                self.push()
        row.restore()

    # Эта операция может произойти в состоянии заёма
    # При повторном заёме произойдёт ошибка
    def sub(self, row):
        assert(type(row) == Row)
        assert(self is not row)
        assert(self.period == row.period)
        assert(len(self.data) < self.period)

        if (row.isEmpty()):
            return
        
        row.preserve()
        while(row.hasItems()):
            row.pop()
            
            if (self.hasItems()):
                self.pop()
            else:
                self.fill()
        row.restore()
    
    def toArray(self):
        assert(len(self.data) < self.period)
        # assert(len(self.data) >= 0)
        arr = np.zeros(self.period - 1, dtype=int)
        self.preserve()
        i = 0
        while(self.hasItems()):
            self.pop()
            arr[i] = 1
            i = i + 1
        self.restore()
        return arr
    
    def setDecimal(self, num):
        assert(type(num) == int)
        # Не будем здесь поднимать флаг заёма
        assert(num < self.period)

        self.clear()
        for i in range(num):
            self.push()

    def getDecimal(self):
        return len(self.data)

In [3]:
r1 = Row(10, 'r1')
r2 = Row(10, 'r2')
r3 = Row(10, 'r3')

r1.setDecimal(2)
r2.setDecimal(4)
r3.setDecimal(7)

assert(r2.isBigger(r1))
assert(r3.isBigger(r2))
assert(not r1.isEquals(r2))
assert(not r1.isBigger(r2))

assert(r1.getDecimal() == 2)
assert(r2.getDecimal() == 4)
assert(r3.getDecimal() == 7)

r2.add(r1)
assert(r2.getDecimal() == 6)
assert(r1.getDecimal() == 2)
assert(not r2.isCarry())

r2.add(r3)
assert(r2.getDecimal() == 3)
assert(r3.getDecimal() == 7)
assert(r2.isCarry())

r3.sub(r2)
assert(r3.getDecimal() == 4)
assert(r2.getDecimal() == 3)
assert(not r3.isCarry())

r2.clearCarry()
r2.sub(r3)
assert(r2.getDecimal() == 9)
assert(r3.getDecimal() == 4)
assert(r2.isCarry())

r2.clearCarry()
r1.setDecimal(5)
r2.setDecimal(7)
r3.setDecimal(7)

assert(r2.isBigger(r1))
assert(r3.isEquals(r2))
assert(not r3.isBigger(r2))

In [4]:
class Abak():
    def __init__(self, shape, name='a'):
        assert(type(shape) == tuple)
        assert(len(shape) == 2)
        periods = shape[0]
        period = shape[1]
        assert(periods > 0)
        assert(period > 0)
        self.name = name
        self.shape = shape
        self.maxValue = (period ** periods) - 1
        self.rows = [Row(period, name) for i in range(periods)]
        self.stack = []

    def preserve(self):
        state = Abak(self.shape, self.name)
        assert(state.isEmpty())
        state.apply(self)
        self.stack.append(state)

    def restore(self):
        assert(len(self.stack) > 0)
        state = self.stack.pop()
        self.apply(state)

    #def stackHasItems(self):
    #    return len(self.stack) > 0
    
    # Принимаем значение переданной досточки
    def apply(self, abak):
        assert(isinstance(abak, Abak))
        assert(self is not abak)
        assert(self.shape[0] >= abak.shape[0])
        assert(self.shape[1] == abak.shape[1])
        self.clear()
        self.add(abak)
    
    # Больше чем переданная таблица
    def isBigger(self, abak):
        assert(isinstance(abak, Abak))
        assert(self is not abak)
        assert(self.shape[1] == abak.shape[1])

        periodSelf = self.shape[0] - 1
        periodOther = abak.shape[0] - 1
        
        while(self.rows[periodSelf].isEmpty() and periodSelf >= 0):
            periodSelf = periodSelf - 1
        while(abak.rows[periodOther].isEmpty() and periodOther >= 0):
            periodOther = periodOther - 1
        
        if (periodSelf < 0 and periodOther < 0):
            return False
        elif (periodSelf > periodOther):
            return True
        elif (periodSelf < periodOther):
            return False
        else:
            rowSelf = self.rows[periodSelf]
            rowOther = abak.rows[periodOther]
            return rowSelf.isBigger(rowOther)

    def isEquals(self, abak):
        assert(isinstance(abak, Abak))
        assert(self is not abak)
        assert(self.shape[0] >= abak.shape[0])
        assert(self.shape[1] == abak.shape[1])

        for i in range(abak.shape[0]):
            if (not self.rows[i].isEquals(abak.rows[i])):
                return False
        if (self.shape[0] == abak.shape[0]):
            return True
        for j in range(i + 1, self.shape[0]):
            if (self.rows[j].hasItems()):
                return False
        return True
    
    def hasItems(self):
        for i in range(self.shape[0]):
            if (self.rows[i].hasItems()):
                return True
        return False
    
    def isEmpty(self):
        for i in range(self.shape[0]):
            if (self.rows[i].hasItems()):
                return False
        return True

    def clear(self):
        for i in range(self.shape[0]):
            self.rows[i].clear()

    def clearCarry(self):
        for i in range(self.shape[0]):
            self.rows[i].clearCarry()
    
    def push(self):
        i = 0
        self.clearCarry()
        self.rows[i].pushOrCarry()
        while(self.rows[i].isCarry()):
            i = i + 1
            assert(i < self.shape[0])
            self.rows[i].pushOrCarry()
        assert(not self.rows[i].isCarry())
        
    def pop(self):
        i = 0
        self.clearCarry()
        self.rows[i].popOrCarry()
        while(self.rows[i].isCarry()):
            i = i + 1
            assert(i < self.shape[0])
            self.rows[i].popOrCarry()
        assert(not self.rows[i].isCarry())

    # Сдвиг влево относительно написания десятичного числа
    # Направление сдвига: от менее значащих разрядов к более значащим
    def shl(self):
        # Относительно модели стека, самый значащий разряд у нас сверху
        row = self.rows.pop()
        assert(row.isEmpty())
        self.rows.insert(0, row)

    # Сдвиг вправо относительно написания десятичного числа
    # Направление сдвига: от более значащих разрядов к менее значащим
    def shr(self, abak):
        assert(isinstance(abak, Abak))
        assert(self.shape[0] >= abak.shape[0])
        assert(self.shape[1] == abak.shape[1])
        # Относительно модели стека, наименее значащий разряд у нас снизу
        row = self.rows.pop(0)
        abak.clear()
        while(row.hasItems()):
            row.pop()
            abak.push()
        row.clear()
        self.rows.append(row)
    
    def add(self, abak):
        assert(isinstance(abak, Abak))
        assert(self.shape[0] >= abak.shape[0])
        assert(self.shape[1] == abak.shape[1])

        self.clearCarry()
        periods = abak.shape[0]
        isCarry = False
        for i in range(periods):
            if (isCarry):
                # Переход в состояние заёма из состояния нулевого баланса
                self.rows[i].pushOrCarry()
            # Переход в состояние заёма из состояния нулевого баланса
            self.rows[i].add(abak.rows[i])
            isCarry = self.rows[i].isCarry()
        for j in range(i + 1, self.shape[0]):
            if (isCarry):
                self.rows[j].pushOrCarry()
                isCarry = self.rows[j].isCarry()
            else:
                break
        assert(not isCarry)
    
    def sub(self, abak):
        assert(type(abak) == Abak)
        assert(self.shape[0] >= abak.shape[0])
        assert(self.shape[1] == abak.shape[1])
        assert(not abak.isBigger(self))

        self.clearCarry()
        periods = abak.shape[0]
        isCarry = False
        for i in range(periods):
            if (isCarry):
                # Переход в состояние заёма из состояния нулевого баланса
                self.rows[i].popOrCarry()
            # Переход в состояние заёма из состояния нулевого баланса
            self.rows[i].sub(abak.rows[i])
            isCarry = self.rows[i].isCarry()
        for j in range(i + 1, self.shape[0]):
            if (isCarry):
                self.rows[j].popOrCarry()
                isCarry = self.rows[j].isCarry()
            else:
                break
        assert(not isCarry)

    def inc(self, right):
        assert(type(right) == Abak)
        assert(self.shape[0] >= right.shape[0])
        assert(self.shape[1] == right.shape[1])
        
        # Умножаем большее на меньшее, или возводим в квадрат
        assert(not right.isBigger(self))
        
        state = Abak(self.shape, self.name)
        assert(state.isEmpty())
        state.apply(self)
        
        right.preserve()
        right.pop()
        while(right.hasItems()):
            self.add(state)
            right.pop()
        right.restore()

    def dec(self, right, rest):
        assert(isinstance(right, Abak))
        assert(isinstance(rest, Abak))
        assert(self.shape[0] >= right.shape[0])
        assert(self.shape[1] == right.shape[1])
        assert(self.shape[0] >= rest.shape[0])
        assert(self.shape[1] == rest.shape[1])
        
        # Делим на себя
        if (self.isEquals(right)):
            self.clear()
            self.push()
            return

        # Делим большее на меньшее
        assert(self.isBigger(right))
        
        rest.apply(self)
        self.clear()
        
        while(rest.isBigger(right)):
            rest.sub(right)
            self.push()

        assert(rest.hasItems())
        if (rest.isEquals(right)):
            rest.sub(right)
            self.push() 
    
    def toFrame(self):
        rows = self.shape[0]
        cols = self.shape[1] - 1
        table = np.zeros((rows, cols), dtype=int)
        for i in range(rows):
            table[i] = self.rows[i].toArray()
        return pd.DataFrame(
            table,
            columns=[str(i + 1) for i in range(cols)]) 

    def setDecimal(self, num):
        assert(type(num) == int)
        assert(num > 0)
        assert(num <= self.maxValue)

        periods = self.shape[0]
        for i in range(periods):
            self.rows[i].clear()
            
        if (num == 0):
            return

        i = 0
        period = self.shape[1]
        # Ищем граничный период
        while((period ** i) <= num):
            i = i + 1

        for j in reversed(range(i)):
            amount = period ** j
            while((num - amount) >= 0):
                self.rows[j].push()
                num = num - amount

    def getDecimal(self):
        periods = self.shape[0]
        period = self.shape[1]

        num = 0
        for i in range(periods):
            amount = self.rows[i].getDecimal()
            num = num + (period ** i) * amount

        return num

In [5]:
# Передаём количество периодов и размер периода
a1 = Abak((4, 10), 'a1')
a2 = Abak((4, 10), 'a2')
a1.setDecimal(100)
a2.setDecimal(4321)
assert(a1.getDecimal() == 100)
assert(a2.getDecimal() == 4321)
assert(a2.isBigger(a1))
assert(not a2.isEquals(a1))
assert(not a1.isBigger(a2))

a2.add(a1)
assert(a2.getDecimal() == 4421)
assert(a1.getDecimal() == 100)

a2.sub(a1)
assert(a2.getDecimal() == 4321)
assert(a1.getDecimal() == 100)

a1.setDecimal(4321)
a2.sub(a1)
assert(a2.getDecimal() == 0)
assert(a1.getDecimal() == 4321)

a1.setDecimal(999)
a1.push()
assert(a1.getDecimal() == 1000)

a1.pop()
assert(a1.getDecimal() == 999)

a1.setDecimal(100)
a2.setDecimal(100)
assert(a1.isEquals(a2))
assert(not a1.isBigger(a2))

a1.setDecimal(4321)
a1.toFrame()

,1,2,3,4,5,6,7,8,9
0,1,0,0,0,0,0,0,0,0
1,1,1,0,0,0,0,0,0,0
2,1,1,1,0,0,0,0,0,0
3,1,1,1,1,0,0,0,0,0


In [6]:
b1 = Abak((3, 10), 'a1')
b2 = Abak((2, 10), 'a2')
b1.setDecimal(12)
b2.setDecimal(12)
assert(b1.isEquals(b2))
assert(not b1.isBigger(b2))
assert(not b2.isBigger(b1))

b1.setDecimal(100)
b2.setDecimal(1)
b1.sub(b2)
assert(b1.getDecimal() == 99)
assert(b2.getDecimal() == 1)

In [7]:
c1 = Abak((3, 2), 'c1')
c2 = Abak((1, 2), 'c2')
c1.setDecimal(1)
assert(c1.getDecimal() == 1)
c1.toFrame()[::-1].T

,2,1,0
1,0,0,1


In [8]:
c1.shl()
c1.shl()
assert(c1.getDecimal() == 4)
c1.toFrame()[::-1].T

,2,1,0
1,1,0,0


In [9]:
c1.shr(c2)
assert(c1.getDecimal() == 2)
assert(c2.getDecimal() == 0)
c1.toFrame()[::-1].T

,2,1,0
1,0,1,0


In [10]:
c1.shr(c2)
c1.shr(c2)
assert(c1.getDecimal() == 0)
assert(c2.getDecimal() == 1)
c1.toFrame()[::-1].T

,2,1,0
1,0,0,0


In [11]:
a1 = Abak((3, 10), 'a1')
a2 = Abak((1, 10), 'a2')
r1 = Abak((3, 10), 'r1')
a1.setDecimal(4)
a2.setDecimal(3)
a1.inc(a2)
assert(a1.getDecimal() == 12)
assert(a2.getDecimal() == 3)
assert(r1.getDecimal() == 0)

a1.setDecimal(4)
a2.setDecimal(4)
a1.dec(a2, r1)
assert(a1.getDecimal() == 1)
assert(a2.getDecimal() == 4)
assert(r1.getDecimal() == 0)

a1.setDecimal(13)
a2.setDecimal(3)
a1.dec(a2, r1)
assert(a1.getDecimal() == 4)
assert(a2.getDecimal() == 3)
assert(r1.getDecimal() == 1)

a1.setDecimal(120)
a2.setDecimal(6)
a1.dec(a2, r1)
assert(a1.getDecimal() == 20)
assert(a2.getDecimal() == 6)
assert(r1.getDecimal() == 0)

a1.toFrame()

,1,2,3,4,5,6,7,8,9
0,0,0,0,0,0,0,0,0,0
1,1,1,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0


In [136]:
class DivMul(Abak):
    def mul(self, right):
        assert(isinstance(right, Abak))
        assert(self.shape[0] >= right.shape[0])
        assert(self.shape[1] == right.shape[1])

        # Умножаем большее на меньшее, или возводим в квадрат
        assert(not right.isBigger(self))

        shape = (1, self.shape[1])
        cell = Abak(shape, self.name)
        state = Abak(self.shape, self.name)
        state.apply(self)
        
        right.preserve()
        right.shr(cell)
        self.inc(cell)

        while(right.hasItems()):
            self.preserve()
            state.shl()
            self.apply(state)
            right.shr(cell)            
            self.inc(cell)

        right.restore()
        right.preserve()

        state.apply(self)
        cell.clear()        
        right.shr(cell)
        
        while(right.hasItems()):
            self.restore()
            state.add(self)
            right.shr(cell)

        self.apply(state)
        right.restore()    

In [141]:
a1 = DivMul((2, 10), 'a1')
a2 = DivMul((1, 10), 'a2')

a1.setDecimal(7)
a2.setDecimal(7)

a1.mul(a2)
assert(a1.getDecimal() == 49)
assert(a2.getDecimal() == 7)

a1 = DivMul((6, 8), 'a1')
a2 = DivMul((3, 8), 'a2')

a1.setDecimal(777)
a2.setDecimal(111)
a1.toFrame()

,1,2,3,4,5,6,7
0,1,0,0,0,0,0,0
1,1,0,0,0,0,0,0
2,1,1,1,1,0,0,0
3,1,0,0,0,0,0,0
4,0,0,0,0,0,0,0
5,0,0,0,0,0,0,0


In [142]:
a2.toFrame()

,1,2,3,4,5,6,7
0,1,1,1,1,1,1,1
1,1,1,1,1,1,0,0
2,1,0,0,0,0,0,0


In [143]:
a1.mul(a2)
assert(a1.getDecimal() == 86247)
assert(a2.getDecimal() == 111)
a1.toFrame()

,1,2,3,4,5,6,7
0,1,1,1,1,1,1,1
1,1,1,1,1,0,0,0
2,1,1,1,0,0,0,0
3,0,0,0,0,0,0,0
4,1,1,1,1,1,0,0
5,1,1,0,0,0,0,0
